# CHANGE DETECTION

## Instalar
Instalar geemap y eeconvert en Google Colab.

In [ ]:
!pip install geemap
!pip install eeconvert

     |████████████████████████████████| 389kB 5.7MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
     |████████████████████████████████| 983kB 24.7MB/s 
     |████████████████████████████████| 5.1MB 33.5MB/s 
     |████████████████████████████████| 225kB 47.4MB/s 
     |████████████████████████████████| 1.9MB 46.7MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 1.1MB 39.9MB/s 
     |████████████████████████████████| 102kB 10.6MB/s 
     |████████████████████████████████| 102kB 10.2MB/s 
     |████████████████████████████████| 112kB 38.3MB/s 
     |████████████████████████████████| 225kB 47.0MB/s 
     |████████████████████████████████| 552kB 42.8MB/s 
     |████████████████████████████████| 122kB 47.0MB/s 
  Created wheel for pyshp: filename=pyshp-2.1.2-cp36-none-any.whl size=36216 sha256=5001cdbc3c89a7ed287cb7a5d1a446bc579665d12ddd7ee4be5e119fbd0c247f
  Stored in directory: /root/.cache/pip/wheels/96/6c/53/4112475adf3b831d

     |████████████████████████████████| 102kB 4.1MB/s 
     |████████████████████████████████| 972kB 11.4MB/s 
     |████████████████████████████████| 7.0MB 38.8MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
     |████████████████████████████████| 6.5MB 38.9MB/s 
     |████████████████████████████████| 14.8MB 169kB/s 
     |████████████████████████████████| 143kB 46.2MB/s 
  Created wheel for boto3: filename=boto3-1.16.27-py2.py3-none-any.whl size=128454 sha256=b758db213344dacf8c1b0a07b07502c557ffee4735ba2023b01a246025c5a2c4
  Stored in directory: /root/.cache/pip/wheels/f7/4a/29/e5b74fd7012b8322191f5db368a910a78b013ce96bf4259d50
Successfully built boto3
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.11.0 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
 

## Importar

- Google Earth Engine y utilidades

In [ ]:
import ee, eeconvert, geemap.eefolium as geemap

- Manejo de (Geo) Data Frames

In [ ]:
import geopandas as gpd, pandas as pd

- Manejo de Arrays

In [ ]:
import numpy as np

## Iniciar GEE
- Autenticación

In [ ]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=GmsVyv1G3S1-BX38bHCQmEcPeKb4rKQBXRSNOjL6FNE&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g4fk2beEkdtn03qc64XaTxF7PSeZkJUNxwxLoSQTDbBaOPlthSMCvQ

Successfully saved authorization token.


- Inicialización

In [ ]:
ee.Initialize()

## Datos

Directorio donde se encuentra el shapefile.

In [ ]:
PATH = '/content/drive/MyDrive/fiverr/2020-11-30 LANDTRENDR REPUBLICA DOMINICANA FRANK/'

Lectura del shapefile y cambio del sistema de referencia.

In [ ]:
points = gpd.read_file(PATH + "Zona_influencia_DR.shp").to_crs("EPSG:4326")

Asignación de un identificador único para cada registro.

In [ ]:
points['ID'] = points['cuest_id_b'].astype(str) + points['parcela'].astype(str)

Obtener sólo los registros corregidos.

In [ ]:
points = points[points['corregido2'].str.contains("ok")][['ID','patca','geometry']]

Convertir a Feature Collection.

In [ ]:
eepoints = eeconvert.gdfToFc(points)

## Preprocesamiento de Imágenes

Nombres en para bandas en común entre Landsat 7 y Landsat 8. Esto con el fin de crear una única colección de imágenes con ambos datasets.

In [ ]:
bandNames = ['BLUE','GREEN','RED','NIR','SWIR1','SWIR2']

Bandas correspondientes para cada satélite.

In [ ]:
L7Bands = ['B1','B2','B3','B4','B5','B7']
L8Bands = ['B2','B3','B4','B5','B6','B7']

Lectura de cada uno de los datasets. Se realizan los filtros correspondientes por ubicación, fecha y se seleccionan únicamente las bandas en común (también se renombran).

In [ ]:
L7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR").filterBounds(eepoints).filterDate('2011-01-01','2020-01-01').select(L7Bands,bandNames)
L8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR").filterBounds(eepoints).filterDate('2011-01-01','2020-01-01').select(L8Bands,bandNames)

Unir ambas colecciones en una sola y ordenar por fecha.

In [ ]:
L78 = L7.merge(L8).sort('system:time_start')

## Detección de Cambios

### Algoritmo Continuous Change Detection and Classification
Este algoritmo recibe una colección de imágenes, utiliza todas las bandas para ajustar funciones armónicas y encontrar fechas de quiebre cuando las funciones cambian (el algoritmo funciona a base de pixel).

> **PUEDE HABER MÁS DE UNA FECHA DE QUIEBRE**

- Se utilizaron todas las bandas para el algoritmo (el algoritmo da la opción de usar sólo las bandas seleccionadas, así que este parámetro puede cambiarse a gusto).
- Se utilizaron las bandas VERDE y SWIR2 para enmascarar nubes y sombras usando el algoritmo TMASK-
- El formato de fecha de salida es MILISEGUNDOS.



In [ ]:
CCDC = ee.Algorithms.TemporalSegmentation.Ccdc(collection = L78,tmaskBands = ['GREEN','SWIR2'],dateFormat = 2)

NameError: ignored

El algoritmo encuentra segmentos y guarda las fechas de acuerdo a segmentos:

- tStart: Inicio de segmento.
- tEnd: Fin de segmento.
- tBreak: Punto de quiebre (inicio del siguiente segmento: siguiente _tStart_. El fin de un segmento _tEnd_ no es necesariamente el inicio de otro segmento _tStart_ y por lo tanto no necesariamente un punto de quiebre _tBreak_).
- numObs: Observaciones del segmento.

Estos valores se extraen por cada registro del Feature Collection. Al ser polígonos, sólo nos interesa un valor (ee.Reducer.first()).

In [ ]:
eepointsCCDC = CCDC.select(['tStart','tEnd','tBreak','numObs']).reduceRegions(collection = eepoints,reducer = ee.Reducer.first(),scale = 30,tileScale = 16)

NameError: ignored

Convertir valores extraídos a un Data Frame.

In [ ]:
pointsCCDC = eeconvert.fcToGdf(eepointsCCDC)

Los valores se guardan en listas (el tamaño de la lista es proporcional al número de segmentos encontrados). Más de un segmento significa que un cambio fue encontrado. Estas listas son explotadas y apiladas en la misma columna.

In [ ]:
tStart = pointsCCDC['tStart'].explode()
tEnd = pointsCCDC['tEnd'].explode()
tBreak = pointsCCDC['tBreak'].explode()
numObs = pointsCCDC['numObs'].explode()

Ya que las listas fueron explotadas y apiladas en la misma columna, los índices de los registros que se apilaron se utilizan para replicar los registros del data frame. Si una lista tiene dos segmentos para un registro, este debe ahora aparecer dos veces.

In [ ]:
pointsExploded = pointsCCDC.loc[numObs.index,['ID','patca','geometry']]

Se añaden los valores de las listas explotadas al data frame donde se replicaron los registros. En el caso de los datos de fecha, estos son formateados de acuerdo a sus unidades de MILISEGUNDOS.

In [ ]:
pointsExploded['tStart'] = pd.to_datetime(tStart,unit = 'ms')
pointsExploded['tEnd'] = pd.to_datetime(tEnd,unit = 'ms')
pointsExploded['tBreak'] = pd.to_datetime(tBreak,unit = 'ms')

En el caso del número de observaciones no se realiza ningún cambio.

In [ ]:
pointsExploded['numObs'] = numObs

Cuando sólo existe un segmento, no existe (generalmente) punto de quiebre y su valor de MILISEGUNDOS es configurado en cero, lo que equivale a una fecha de 1970 (esto también ocurre a veces en el segmento final cuando hay más de un segmento). Así que podemos cambiar estos valores para evitar confusiones.

In [ ]:
pointsExploded.loc[pointsExploded['tBreak'] < '2011-01-01','tBreak'] = np.nan

## Ejemplos

El registro con ID 386311 sólo tiene un segmento de 131 imágenes que inicia en 2011-01-01 y termina en 20019-12-17. No se ecnontró ningún punto de quiebre o cambio de cobertura en las imágenes.

In [ ]:
pointsExploded[pointsExploded['ID'] == '386311']

,ID,patca,geometry,tStart,tEnd,tBreak,numObs
0,386311,1,"POLYGON ((-71.70753 18.89259, -71.70756 18.892...",2011-01-01 15:01:56.470,2019-12-17 15:09:03.062,NaT,131


El registro con ID 396011 tiene 4 segmentos con 47, 42, 31 y 6 imágenes respectivamente. El primer segmento inicia en 2011-01-01 y termina en 2015-03-25. El segundo segmento inicia en 2015-04-10 y termina en 2018-01-28. Y así sucesivamente.

In [ ]:
pointsExploded[pointsExploded['ID'] == '396011']

,ID,patca,geometry,tStart,tEnd,tBreak,numObs
2,396011,0,"POLYGON ((-70.82647 18.39846, -70.82666 18.398...",2011-01-01 15:01:56.470,2015-03-25 15:08:16.830,2015-04-10 15:08:08.210,47
2,396011,0,"POLYGON ((-70.82647 18.39846, -70.82666 18.398...",2015-04-10 15:08:08.210,2018-01-28 15:08:41.000,2018-02-13 15:08:34.810,42
2,396011,0,"POLYGON ((-70.82647 18.39846, -70.82666 18.398...",2018-02-13 15:08:34.810,2019-09-12 15:09:01.304,2019-09-28 15:09:06.465,31
2,396011,0,"POLYGON ((-70.82647 18.39846, -70.82666 18.398...",2019-09-28 15:09:06.465,2019-12-17 15:09:03.062,NaT,6
